<a href="https://colab.research.google.com/github/iyadaithou/Fallacy-Recognition/blob/main/Preliminary_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers spacy
!python -m spacy download en_core_web_md
!pip install sentence-transformers


In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("tasksource/logical-fallacy")
# Print the structure of the dataset
print(dataset)

# Access a specific split
train_dataset = dataset['train']

# Print the first few examples from the training set
print(train_dataset[:5])

DatasetDict({
    train: Dataset({
        features: ['config', 'source_article', 'logical_fallacies'],
        num_rows: 2680
    })
    test: Dataset({
        features: ['config', 'source_article', 'logical_fallacies'],
        num_rows: 511
    })
    dev: Dataset({
        features: ['config', 'source_article', 'logical_fallacies'],
        num_rows: 570
    })
})
{'config': ['edu', 'edu', 'edu', 'edu', 'edu'], 'source_article': ['company\'s slogan "Expect More. Pay Less."', "The bigger a child's shoe size, the better the child's handwriting", 'Since many people believe this, then it must be true', "Senator Randall isn't lying when she says she cares about her constituents—she wouldn't lie to people she cares about.", 'A mother is telling her daughter that she went over her data for the month, and the daughter begins telling her mother about getting an A on a math test.'], 'logical_fallacies': ['appeal to emotion', 'false causality', 'ad populum', 'circular reasoning', 'fallacy of

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd



# Step 1 & 2: Data is already split

# Prepare the data (assuming dataset is already loaded)
train_texts = dataset['train']['source_article']
train_labels = dataset['train']['logical_fallacies']
dev_texts = dataset['dev']['source_article']
dev_labels = dataset['dev']['logical_fallacies']
test_texts = dataset['test']['source_article']
test_labels = dataset['test']['logical_fallacies']

# Step 4: Encode labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
dev_labels_encoded = label_encoder.transform(dev_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Step 3 & 5: Create a pipeline with TF-IDF and Logistic Regression
model = make_pipeline(TfidfVectorizer(lowercase=True, stop_words='english'), LogisticRegression(max_iter=1000))

# Train the model
model.fit(train_texts, train_labels_encoded)

# Step 6: Evaluate the model
dev_predictions = model.predict(dev_texts)
print("Validation Set Performance:")
print(classification_report(dev_labels_encoded, dev_predictions, target_names=label_encoder.classes_))

# Finally, evaluate on the test set
test_predictions = model.predict(test_texts)
print("Test Set Performance:")
print(classification_report(test_labels_encoded, test_predictions, target_names=label_encoder.classes_))


# Define a function to calculate metrics and return them in a dictionary
def calculate_metrics(y_true, y_pred, average_type='weighted'):
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average=average_type)
    return {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1 Score': f1}

# Calculate metrics for the test set
test_metrics = calculate_metrics(test_labels_encoded, test_predictions)

# Add the metrics to a new dataframe row
results_df = pd.DataFrame([test_metrics], index=['Logistic Regression'])

# Display the dataframe
print(results_df)



Validation Set Performance:
                        precision    recall  f1-score   support

            ad hominem       0.42      0.46      0.44        52
            ad populum       0.50      0.29      0.37        51
     appeal to emotion       0.36      0.21      0.27        42
    circular reasoning       0.67      0.33      0.44        18
          equivocation       0.00      0.00      0.00         9
fallacy of credibility       0.38      0.12      0.18        26
  fallacy of extension       0.50      0.07      0.13        40
      fallacy of logic       0.00      0.00      0.00        33
  fallacy of relevance       0.83      0.11      0.20        44
       false causality       0.47      0.19      0.27        43
         false dilemma       0.71      0.17      0.28        29
 faulty generalization       0.28      0.67      0.39        84
           intentional       0.31      0.64      0.42        99

              accuracy                           0.35       570
          

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

!pip install accelerate -U
!pip install transformers[torch] -U


# Load the dataset
dataset = load_dataset("tasksource/logical-fallacy")

# Initialize the tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Tokenize the input (this will take care of padding/truncation)
def tokenize_function(examples):
    return tokenizer(examples['source_article'], padding='max_length', truncation=True, max_length=512)

# Apply the tokenizer to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Initialize LabelEncoder
label_encoder = LabelEncoder()
# Fit the label encoder using the logical fallacies labels from the train split
label_encoder.fit(dataset['train']['logical_fallacies'])
# Create a dictionary that maps each label to an integer
label2id = {label: id for id, label in enumerate(label_encoder.classes_)}
# Create a dictionary that maps each integer to its corresponding label
id2label = {id: label for label, id in label2id.items()}

# Function to encode labels
def encode_labels(example):
    example['labels'] = label2id[example['logical_fallacies']]
    return example

# Encode the labels in all datasets
tokenized_datasets = tokenized_datasets.map(encode_labels)

# Load the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label2id))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy='epoch',
)

# Function to compute metrics for evaluation
def compute_metrics(p):
    predictions = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['dev'],
    compute_metrics=compute_metrics
    )

trainer.train()

# Evaluate the model
test_results = trainer.evaluate(tokenized_datasets['test'])

# Convert the results to a pandas dataframe
results_df = pd.DataFrame([test_results], index=['DistilBERT Model'])
results_df = results_df.drop(columns=['eval_loss', 'epoch'])
print(results_df)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

# Evaluate the model
test_results = trainer.evaluate(tokenized_datasets['test'])

# Convert the results to a pandas dataframe
results_df = pd.DataFrame([test_results], index=['DistilBERT Model'])
results_df = results_df.drop(columns=['eval_loss', 'epoch'])
print(results_df)

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,2.345141,0.249123,0.143465,0.113262,0.249123
2,No log,1.960727,0.394737,0.358174,0.513196,0.394737
3,2.091700,1.750834,0.438596,0.429805,0.459586,0.438596


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                  eval_accuracy   eval_f1  eval_precision  eval_recall  \
DistilBERT Model       0.399217  0.397622        0.447941     0.399217   

                  eval_runtime  eval_samples_per_second  eval_steps_per_second  
DistilBERT Model       31.7317                   16.104                  2.017  


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import spacy
from transformers import AutoTokenizer, AutoModel
import torch

# Load models
nlp = spacy.load("en_core_web_md")
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# Define amplifiers with assigned strength
amplifiers = {"some": 1, "a few": 1, "several": 2, "many": 3, "most": 4, "all": 5, "every": 5, "always": 5}

def calculate_semantic_similarity(sentence1, sentence2):
    inputs1 = tokenizer(sentence1, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs2 = tokenizer(sentence2, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs1 = model(**inputs1)
        outputs2 = model(**inputs2)
    sentence_embedding1 = outputs1.last_hidden_state.mean(dim=1)
    sentence_embedding2 = outputs2.last_hidden_state.mean(dim=1)
    cosine_sim = torch.nn.functional.cosine_similarity(sentence_embedding1, sentence_embedding2)
    return cosine_sim.item()

def detect_amplifier_strength(sentence):
    doc = nlp(sentence)
    strength = 0
    for token in doc:
        for amp, val in amplifiers.items():
            if amp in token.text:
                strength = max(strength, val)
    return strength

def amplification_factor(sentence1, sentence2):
    semantic_similarity = calculate_semantic_similarity(sentence1, sentence2)
    # If the sentences are identical, set amplification factor to 0
    if semantic_similarity == 1:
        return 0

    strength1 = detect_amplifier_strength(sentence1)
    strength2 = detect_amplifier_strength(sentence2)

    # Calculate the amplification factor based on the presence and change in amplifier strength
    if strength2 > strength1:
        # Normalize the change in strength to a scale of 0 to 1
        change_in_strength = (strength2 - strength1) / max(amplifiers.values())
        # Factor in semantic similarity to ensure relevance
        factor = min(change_in_strength * semantic_similarity, 1.0)
    else:
        # No amplification or semantic relevance
        factor = semantic_similarity * 0.1  # Slight adjustment to account for some relevance

    return factor

# Example usage
sentence1 = "Some students passed the exam."
sentence2 = "Some students passed all the exams."
print("Amplification Factor:", amplification_factor(sentence1, sentence2))

sentence1 = "All students passed the exam."
sentence2 = "some students passed the exam."
print("Amplification Factor:", amplification_factor(sentence1, sentence2))


Amplification Factor: 0.9350095987319946
Amplification Factor: 0.1872041463851929
